In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pylab as plt
import json
import time
import warnings

from bs4 import BeautifulSoup
from tqdm import tqdm
import requests

from joblib import Parallel, delayed
from tqdm_joblib import tqdm_joblib
import plotly.express as px
from tqdm.autonotebook import tqdm

import openpyxl 
import pycountry

import nltk
#nltk.download()


warnings.filterwarnings('ignore')


c:\Users\javir\anaconda3\envs\ETL\lib\site-packages\tqdm_joblib\__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
titles = pd.read_csv('../data/clean/titles.csv', encoding='utf-8', encoding_errors='ignore')
comments = pd.read_csv('../data/clean/com_group.csv', encoding='utf-8', encoding_errors='ignore')

## Recomendación por pelicula

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words= 'english')      # Definir objeto vectorizador TF_IDF

In [4]:

tfidf_matrix = tfidf.fit_transform(titles['description'])      # contruir matriz TF-IDF
tfidf_matrix.shape

(15316, 37380)

In [5]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim= linear_kernel(tfidf_matrix, tfidf_matrix)       #similitud de cosenos

In [6]:
# construir mapa inverso de indices y titulos de peliculas

indices= pd.Series(titles.index, index= titles['title']).drop_duplicates()

In [11]:
def get_recom (title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores= list(enumerate(cosine_sim[idx]))

    sim_scores= sorted(sim_scores, key= lambda x: x[1], reverse=True)

    #sim_scores=  sim_scores[1:11]

    movie_id= [i[0]for i in sim_scores]

    df_filtrado = titles.iloc[movie_id]

    return df_filtrado

In [7]:
title = "Breaking Bad"

idx = indices[title]
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
movie_id = [i[0] for i in sim_scores]
df_filtrado = titles.iloc[movie_id]
df_filtrado = df_filtrado.iloc[1:]



In [8]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
2876,tm455568,Paddleton,MOVIE,An unlikely friendship between two neighbors b...,2019,R,89,"drama, comedy",United States,-1.0,tt8041276,7.1,13791,7339,7.0,Netflix,"Mark Duplass, Ray Romano, Christine Woods, Jen..."
3005,tm818502,"Live Twice, Love Once",MOVIE,A retired academic teacher tries to find the l...,2019,unknown,101,"comedy, drama, european",Spain,-1.0,tt9063902,7.2,6588,14209,7.5,Netflix,"Inma Cuesta, Oscar Martínez, Mafalda Carbonell..."
6216,tm38781,Wit,MOVIE,A renowned professor is forced to reassess her...,2001,PG-13,99,"drama, documentation, european",United States,-1.0,tt0243664,8.0,10993,12739,7.3,HBO,"Emma Thompson, Christopher Lloyd, Eileen Atkin..."
9378,tm192032,The Ride,MOVIE,A has-been cowboy is given a second chance at ...,1997,PG,101,"drama, family, sport, western",United States,-1.0,tt0127247,6.6,494,3595,6.8,Amazon,"Michael Biehn, Clarence Felder, Brock Pierce, ..."
12645,tm448634,Say Yes,MOVIE,A young woman diagnosed with cancer tries to p...,2018,unknown,94,drama,United States,-1.0,tt7217362,6.5,538,2848,4.8,Amazon,"Leah McKendrick, Patrick Zeller, Matt Pascua, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15309,tm1294232,Thankam,MOVIE,Muthu and Kannan are Gold Agents from Thrissur...,2023,unknown,145,"drama, crime",India,-1.0,tt11086194,7.3,2868,5722,8.8,Amazon,"Biju Menon, Vineeth Sreenivasan, Aparna Balamu..."
15310,tm1291792,New Year’s Special,MOVIE,Ingrid Guimarães and Lázaro Ramos hit the grou...,2023,unknown,56,unknown,Brazil,-1.0,tt25753082,6.3,20,675,5.0,Amazon,"Ingrid Guimarães, Lázaro Ramos, Juliette, Maír..."
15311,tm948025,Hard Luck Love Song,MOVIE,"Jesse, a charismatic but down on his luck trou...",2023,R,104,"drama, thriller, romance, music",United States,-1.0,tt8709338,5.3,444,4299,5.3,Amazon,"Michael Dorman, Sophia Bush, RZA, Dermot Mulro..."
15313,ts382337,Marc Marquez: ALL IN,SHOW,Marc Márquez is facing the toughest decision o...,2023,unknown,33,documentation,Spain,1.0,tt26762647,8.1,74,2893,6.0,Amazon,"Marc Márquez, Dani Pedrosa, Jorge Lorenzo, Ale..."


## Prueba

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [6]:
pregunta1= 'Breaking Bad'
pregunta2= 'Misterio'
pregunta3= ''
pregunta4= 'Hechos reales'
#pregunta5= ('0','300')
pregunta6= 'Mucho'
pregunta7= 'hero'
pregunta8= 'Netflix'

In [7]:
respuestas = {
    'pregunta1': pregunta1,
    'pregunta2': pregunta2,
    'pregunta3': pregunta3,
    'pregunta4': pregunta4,
    #'pregunta5': (duracion_minima, duracion_maxima),
    'pregunta6': pregunta6,
    'pregunta7': pregunta7,
    'pregunta8': pregunta8,
}

In [8]:
titles = pd.read_csv('../data/clean/titles.csv', encoding='utf-8', encoding_errors='ignore')
comments = pd.read_csv('../data/clean/com_group.csv', encoding='utf-8', encoding_errors='ignore')

    # 1. Filtro por peli o serie fav
tfidf = TfidfVectorizer(stop_words= 'english')              # Definir objeto vectorizador TF_IDF
tfidf_matrix = tfidf.fit_transform(titles['description'])   # contruir matriz TF-IDF

cosine_sim= linear_kernel(tfidf_matrix, tfidf_matrix)       # similitud de cosenos

indices= pd.Series(titles.index, index= titles['title']).drop_duplicates()  # construir mapa inverso de indices y titulos de peliculas

    #title = "Breaking Bad"

idx = indices[respuestas['pregunta1']]
sim_scores = list(enumerate(cosine_sim[idx]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
movie_id = [i[0] for i in sim_scores]
df_filtrado = titles.iloc[movie_id]
df_filtrado = df_filtrado.iloc[1:]

In [9]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
2876,tm455568,Paddleton,MOVIE,An unlikely friendship between two neighbors b...,2019,R,89,"drama, comedy",United States,-1.0,tt8041276,7.1,13791,7339,7.0,Netflix,"Mark Duplass, Ray Romano, Christine Woods, Jen..."
3005,tm818502,"Live Twice, Love Once",MOVIE,A retired academic teacher tries to find the l...,2019,unknown,101,"comedy, drama, european",Spain,-1.0,tt9063902,7.2,6588,14209,7.5,Netflix,"Inma Cuesta, Oscar Martínez, Mafalda Carbonell..."
6216,tm38781,Wit,MOVIE,A renowned professor is forced to reassess her...,2001,PG-13,99,"drama, documentation, european",United States,-1.0,tt0243664,8.0,10993,12739,7.3,HBO,"Emma Thompson, Christopher Lloyd, Eileen Atkin..."
9378,tm192032,The Ride,MOVIE,A has-been cowboy is given a second chance at ...,1997,PG,101,"drama, family, sport, western",United States,-1.0,tt0127247,6.6,494,3595,6.8,Amazon,"Michael Biehn, Clarence Felder, Brock Pierce, ..."
12645,tm448634,Say Yes,MOVIE,A young woman diagnosed with cancer tries to p...,2018,unknown,94,drama,United States,-1.0,tt7217362,6.5,538,2848,4.8,Amazon,"Leah McKendrick, Patrick Zeller, Matt Pascua, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15309,tm1294232,Thankam,MOVIE,Muthu and Kannan are Gold Agents from Thrissur...,2023,unknown,145,"drama, crime",India,-1.0,tt11086194,7.3,2868,5722,8.8,Amazon,"Biju Menon, Vineeth Sreenivasan, Aparna Balamu..."
15310,tm1291792,New Year’s Special,MOVIE,Ingrid Guimarães and Lázaro Ramos hit the grou...,2023,unknown,56,unknown,Brazil,-1.0,tt25753082,6.3,20,675,5.0,Amazon,"Ingrid Guimarães, Lázaro Ramos, Juliette, Maír..."
15311,tm948025,Hard Luck Love Song,MOVIE,"Jesse, a charismatic but down on his luck trou...",2023,R,104,"drama, thriller, romance, music",United States,-1.0,tt8709338,5.3,444,4299,5.3,Amazon,"Michael Dorman, Sophia Bush, RZA, Dermot Mulro..."
15313,ts382337,Marc Marquez: ALL IN,SHOW,Marc Márquez is facing the toughest decision o...,2023,unknown,33,documentation,Spain,1.0,tt26762647,8.1,74,2893,6.0,Amazon,"Marc Márquez, Dani Pedrosa, Jorge Lorenzo, Ale..."


## Test

In [9]:
# porcentaje de nulos de la tabla Titles
nan_cols = titles.isna().mean() * 100 
nan_cols[nan_cols>0]

Series([], dtype: float64)

In [10]:
# porcentaje de nulos de la tabla Titles
nan_cols = comments.isna().mean() * 100 
nan_cols[nan_cols>0]

title    2.233351
dtype: float64

In [11]:
pregunta1= 'Contemporáneas'
#pregunta2= 'Acción'
pregunta3= 'Misterio'
pregunta4= ''
pregunta5= 'Hechos reales'
pregunta6= ('0','300')
pregunta7= 'Mucho'
pregunta8= 'hero'
pregunta9= 'Netflix'



In [12]:
respuestas = {
    'pregunta1': pregunta1,
    #'pregunta2': pregunta2,
    'pregunta3': pregunta3,
    'pregunta4': pregunta4,
    'pregunta5': pregunta5,
    #'pregunta6': (duracion_minima, duracion_maxima),
    'pregunta7': pregunta7,
    'pregunta8': pregunta8,
    'pregunta9': pregunta9,
}

In [13]:
# 1. Filtro por antiguedad
if respuestas['pregunta1'] == 'Clásicas':
    df_filtrado = df_filtrado[df_filtrado['release_year'] <= 1990]
elif respuestas['pregunta1'] == 'Contemporáneas':
    df_filtrado = df_filtrado[df_filtrado['release_year'] > 1990]
elif respuestas['pregunta1'] == '¯\_(ツ)_/¯':
    pass

In [14]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
2876,tm455568,Paddleton,MOVIE,An unlikely friendship between two neighbors b...,2019,R,89,"drama, comedy",United States,-1.0,tt8041276,7.1,13791,7339,7.0,Netflix,"Mark Duplass, Ray Romano, Christine Woods, Jen..."
3005,tm818502,"Live Twice, Love Once",MOVIE,A retired academic teacher tries to find the l...,2019,unknown,101,"comedy, drama, european",Spain,-1.0,tt9063902,7.2,6588,14209,7.5,Netflix,"Inma Cuesta, Oscar Martínez, Mafalda Carbonell..."
6216,tm38781,Wit,MOVIE,A renowned professor is forced to reassess her...,2001,PG-13,99,"drama, documentation, european",United States,-1.0,tt0243664,8.0,10993,12739,7.3,HBO,"Emma Thompson, Christopher Lloyd, Eileen Atkin..."
9378,tm192032,The Ride,MOVIE,A has-been cowboy is given a second chance at ...,1997,PG,101,"drama, family, sport, western",United States,-1.0,tt0127247,6.6,494,3595,6.8,Amazon,"Michael Biehn, Clarence Felder, Brock Pierce, ..."
12645,tm448634,Say Yes,MOVIE,A young woman diagnosed with cancer tries to p...,2018,unknown,94,drama,United States,-1.0,tt7217362,6.5,538,2848,4.8,Amazon,"Leah McKendrick, Patrick Zeller, Matt Pascua, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15309,tm1294232,Thankam,MOVIE,Muthu and Kannan are Gold Agents from Thrissur...,2023,unknown,145,"drama, crime",India,-1.0,tt11086194,7.3,2868,5722,8.8,Amazon,"Biju Menon, Vineeth Sreenivasan, Aparna Balamu..."
15310,tm1291792,New Year’s Special,MOVIE,Ingrid Guimarães and Lázaro Ramos hit the grou...,2023,unknown,56,unknown,Brazil,-1.0,tt25753082,6.3,20,675,5.0,Amazon,"Ingrid Guimarães, Lázaro Ramos, Juliette, Maír..."
15311,tm948025,Hard Luck Love Song,MOVIE,"Jesse, a charismatic but down on his luck trou...",2023,R,104,"drama, thriller, romance, music",United States,-1.0,tt8709338,5.3,444,4299,5.3,Amazon,"Michael Dorman, Sophia Bush, RZA, Dermot Mulro..."
15313,ts382337,Marc Marquez: ALL IN,SHOW,Marc Márquez is facing the toughest decision o...,2023,unknown,33,documentation,Spain,1.0,tt26762647,8.1,74,2893,6.0,Amazon,"Marc Márquez, Dani Pedrosa, Jorge Lorenzo, Ale..."


In [15]:
# 3. Filtro por tipo de trama favorita
sinonimos_misterio = ['mystery', 'enigma', 'puzzle', 'riddle', 'conundrum', 'secret', 'intrigue', 'clue', 'suspense']
sinonimos_aventura = ['thrill', 'adventure', 'expedition', 'journey', 'quest', 'explor', 'trek', 'voyage', 'clue', 'safari', 'exploration']
sinonimos_fantasia = ['fantasy', 'imagination', 'imaginary', 'enchant', 'magic', 'tail', 'fairy', 'myth', 'wonder', 'dream', 'illusion', 'super', 'superhero']

def buscar_sinonimos(critica, sinonimos):
    for sinonimo in sinonimos:
        if sinonimo in critica:
            return True
    return False

if respuestas['pregunta3'] == 'Misterio':
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_misterio))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta3'] == 'Aventura':
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_aventura))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta3'] == 'Fantasía':
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_fantasia))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta3'] == '¯\_(ツ)_/¯':
    pass

In [16]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm38781,Wit,MOVIE,A renowned professor is forced to reassess her...,2001,PG-13,99,"drama, documentation, european",United States,-1.0,tt0243664,8.0,10993,12739,7.3,HBO,"Emma Thompson, Christopher Lloyd, Eileen Atkin..."
1,tm31564,Linewatch,MOVIE,"Academy Award Winner Cuba Gooding, Jr. is a Bo...",2008,R,85,"thriller, crime, drama",United States,-1.0,tt1047832,5.4,2884,3972,5.2,Netflix,"Cuba Gooding Jr., Sharon Leal, Dean Norris, Om..."
2,ts57464,Life Sentence,SHOW,When Stella finds out her terminal cancer is c...,2018,TV-14,42,"comedy, drama",United States,1.0,tt6088922,6.4,3899,7925,6.5,Netflix,"Lucy Hale, Elliot Knight, Brooke Lyons, Jayson..."
3,tm374216,Take Your Pills,MOVIE,"In a hypercompetitive world, drugs like Addera...",2018,unknown,87,documentation,United States,-1.0,tt7983844,6.3,5151,11496,6.6,Netflix,"Eben Britton, Dr. Wendy Brown, Alison Klayman"
4,tm700,"10,000 BC",MOVIE,A prehistoric epic that follows a young mammot...,2008,PG-13,109,"action, drama, fantasy, history, thriller","United States, South Africa",-1.0,tt0443649,5.1,130813,24671,5.4,"Netflix, HBO","Steven Strait, Camilla Belle, Cliff Curtis, Na..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6272,tm1297718,Die Hart,MOVIE,"Follows a fictionalized version of Kevin Hart,...",2023,R,85,"action, comedy, thriller",United States,-1.0,tt26198528,4.7,2380,3187531,6.3,Amazon,"Kevin Hart, John Travolta, Nathalie Emmanuel, ..."
6273,ts376700,Cosmic Love France,SHOW,Cosmic Love is the first dating show based on ...,2023,unknown,39,reality,France,1.0,tt24876398,4.6,17,5597,4.0,Amazon,Nabilla Benattia
6274,ts365330,Cinema Marte Dum Tak,SHOW,"J Neelam, Vinod Talwar, Dilip Gulati and Kisha...",2023,unknown,37,documentation,India,1.0,tt21908844,7.6,335,2457,2.0,Amazon,"Vinod Talwar, Dilip Gulati, Kishan Shah, Jaspa..."
6275,tm1302281,Christopher,MOVIE,"A maverick, vigilante cop who is forced to mis...",2023,R,150,"thriller, action, crime",India,-1.0,tt21819994,7.3,1708,362,8.0,Amazon,"Mammootty, Sneha, Amala Paul, Aishwarya Lekshm..."


In [17]:
# 4. Filtro por actor o actriz favorito
if respuestas['pregunta4'] != '':
    actor_favorito = respuestas['pregunta4']
    df_filtrado = df_filtrado[df_filtrado['actors'].str.contains(actor_favorito, case=False)]
else:
    pass

In [18]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm38781,Wit,MOVIE,A renowned professor is forced to reassess her...,2001,PG-13,99,"drama, documentation, european",United States,-1.0,tt0243664,8.0,10993,12739,7.3,HBO,"Emma Thompson, Christopher Lloyd, Eileen Atkin..."
1,tm31564,Linewatch,MOVIE,"Academy Award Winner Cuba Gooding, Jr. is a Bo...",2008,R,85,"thriller, crime, drama",United States,-1.0,tt1047832,5.4,2884,3972,5.2,Netflix,"Cuba Gooding Jr., Sharon Leal, Dean Norris, Om..."
2,ts57464,Life Sentence,SHOW,When Stella finds out her terminal cancer is c...,2018,TV-14,42,"comedy, drama",United States,1.0,tt6088922,6.4,3899,7925,6.5,Netflix,"Lucy Hale, Elliot Knight, Brooke Lyons, Jayson..."
3,tm374216,Take Your Pills,MOVIE,"In a hypercompetitive world, drugs like Addera...",2018,unknown,87,documentation,United States,-1.0,tt7983844,6.3,5151,11496,6.6,Netflix,"Eben Britton, Dr. Wendy Brown, Alison Klayman"
4,tm700,"10,000 BC",MOVIE,A prehistoric epic that follows a young mammot...,2008,PG-13,109,"action, drama, fantasy, history, thriller","United States, South Africa",-1.0,tt0443649,5.1,130813,24671,5.4,"Netflix, HBO","Steven Strait, Camilla Belle, Cliff Curtis, Na..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6272,tm1297718,Die Hart,MOVIE,"Follows a fictionalized version of Kevin Hart,...",2023,R,85,"action, comedy, thriller",United States,-1.0,tt26198528,4.7,2380,3187531,6.3,Amazon,"Kevin Hart, John Travolta, Nathalie Emmanuel, ..."
6273,ts376700,Cosmic Love France,SHOW,Cosmic Love is the first dating show based on ...,2023,unknown,39,reality,France,1.0,tt24876398,4.6,17,5597,4.0,Amazon,Nabilla Benattia
6274,ts365330,Cinema Marte Dum Tak,SHOW,"J Neelam, Vinod Talwar, Dilip Gulati and Kisha...",2023,unknown,37,documentation,India,1.0,tt21908844,7.6,335,2457,2.0,Amazon,"Vinod Talwar, Dilip Gulati, Kishan Shah, Jaspa..."
6275,tm1302281,Christopher,MOVIE,"A maverick, vigilante cop who is forced to mis...",2023,R,150,"thriller, action, crime",India,-1.0,tt21819994,7.3,1708,362,8.0,Amazon,"Mammootty, Sneha, Amala Paul, Aishwarya Lekshm..."


In [19]:

# 6. Filtro para determinar ficción
palabras_clave_ficcion = ['fiction', 'imaginary', 'fantasy', 'fictional', 'otherworld', 'extraterrestrial']
    
if respuestas['pregunta5'] == "Ficción":
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, palabras_clave_ficcion))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta5'] == "Hechos reales":
    comentarios_filtrados = comments[~comments['review'].apply(lambda x: buscar_sinonimos(x, palabras_clave_ficcion))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta5'] == '¯\_(ツ)_/¯':
    pass


In [20]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm38781,Wit,MOVIE,A renowned professor is forced to reassess her...,2001,PG-13,99,"drama, documentation, european",United States,-1.0,tt0243664,8.0,10993,12739,7.3,HBO,"Emma Thompson, Christopher Lloyd, Eileen Atkin..."
1,tm31564,Linewatch,MOVIE,"Academy Award Winner Cuba Gooding, Jr. is a Bo...",2008,R,85,"thriller, crime, drama",United States,-1.0,tt1047832,5.4,2884,3972,5.2,Netflix,"Cuba Gooding Jr., Sharon Leal, Dean Norris, Om..."
2,ts57464,Life Sentence,SHOW,When Stella finds out her terminal cancer is c...,2018,TV-14,42,"comedy, drama",United States,1.0,tt6088922,6.4,3899,7925,6.5,Netflix,"Lucy Hale, Elliot Knight, Brooke Lyons, Jayson..."
3,tm374216,Take Your Pills,MOVIE,"In a hypercompetitive world, drugs like Addera...",2018,unknown,87,documentation,United States,-1.0,tt7983844,6.3,5151,11496,6.6,Netflix,"Eben Britton, Dr. Wendy Brown, Alison Klayman"
4,tm372891,Chris Rock: Tamborine,MOVIE,Chris Rock takes the stage for his first comed...,2018,unknown,64,"comedy, documentation",United States,-1.0,tt8005338,6.6,6982,7743,6.5,Netflix,"Chris Rock, Bo Burnham"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3828,tm1297718,Die Hart,MOVIE,"Follows a fictionalized version of Kevin Hart,...",2023,R,85,"action, comedy, thriller",United States,-1.0,tt26198528,4.7,2380,3187531,6.3,Amazon,"Kevin Hart, John Travolta, Nathalie Emmanuel, ..."
3829,ts376700,Cosmic Love France,SHOW,Cosmic Love is the first dating show based on ...,2023,unknown,39,reality,France,1.0,tt24876398,4.6,17,5597,4.0,Amazon,Nabilla Benattia
3830,ts365330,Cinema Marte Dum Tak,SHOW,"J Neelam, Vinod Talwar, Dilip Gulati and Kisha...",2023,unknown,37,documentation,India,1.0,tt21908844,7.6,335,2457,2.0,Amazon,"Vinod Talwar, Dilip Gulati, Kishan Shah, Jaspa..."
3831,tm1302281,Christopher,MOVIE,"A maverick, vigilante cop who is forced to mis...",2023,R,150,"thriller, action, crime",India,-1.0,tt21819994,7.3,1708,362,8.0,Amazon,"Mammootty, Sneha, Amala Paul, Aishwarya Lekshm..."


In [21]:
# 6. Filtro por duración de la película
#df_filtrado = df_filtrado[(df_filtrado['duration'] >= duracion_minima) & (df_filtrado['duration'] <= duracion_maxima)]

In [22]:
# 7. Filtro por final feliz
sinonimos_final_feliz = ['happy ending', 'positive outcome', 'pleasant conclusion', 'satisfying resolution', 'joyful finale', 'contented ending', 'delightful outcome', 'cheerful conclusion', 'pleasant ending', 'uplifting finale']
    
if respuestas['pregunta7'] == 'Mucho':
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_final_feliz))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta7'] == 'No':
    comentarios_filtrados = comments[~comments['review'].apply(lambda x: buscar_sinonimos(x, sinonimos_final_feliz))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
elif respuestas['pregunta7'] == '¯\_(ツ)_/¯':
    pass

In [23]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm1179579,In Good Hands,MOVIE,"Diagnosed with a terminal illness, a single mo...",2022,unknown,104,"drama, comedy, romance, family",Turkey,-1.0,tt14898794,5.6,2618,829,6.7,Netflix,"Aslı Enver, Kaan Urgancıoğlu, Mert Ege Ak, Ezg..."
1,ts328194,My Name,SHOW,"Following her father's murder, a revenge-drive...",2021,TV-MA,50,"thriller, action, crime, drama","Korea, Republic of",1.0,tt12940504,7.8,19737,86286,8.3,Netflix,"Han So-hee, Park Hee-soon, Ahn Bo-hyun, Kim Sa..."
2,tm974552,Last Three Days,MOVIE,After getting mixed up with a dangerous crime ...,2020,R,87,"thriller, action, crime",United States,-1.0,tt6794504,3.6,573,692,6.7,Amazon,"Robert Palmer Watkins, Thomas Wilson Brown, De..."
3,ts297675,Sexy Beasts,SHOW,"Hoping to say goodbye to superficial dating, r...",2021,TV-14,23,"reality, romance",unknown,2.0,tt14916746,4.6,1133,36721,5.9,Netflix,Rob Delaney
4,tm171793,Monsieur Lazhar,MOVIE,"During a harsh Montréal winter, an elementary-...",2012,PG-13,94,"drama, comedy",Canada,-1.0,tt2011971,7.5,20735,7152,7.1,Amazon,"Mohamed Fellag, Émilien Néron, Danielle Proulx..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,tm1031371,One and the Same,MOVIE,"When a young woman, Lola, is invited to a secr...",2021,unknown,110,scifi,United States,-1.0,tt8906844,4.3,4965,1011,5.7,Amazon,"Acoryé White, Glenn Plummer, Ignacyo Matynia, ..."
294,tm552220,The Blackout Experiment,MOVIE,6 strangers wake up inside a room garnished wi...,2021,R,81,"horror, thriller, drama",United States,-1.0,tt3659284,2.9,239,6395,5.0,Amazon,"Yasmin Irvine, Stefani Rose, Uli Auliani, Chev..."
295,tm822572,Gun and a Hotel Bible,MOVIE,"Based on the original play, Gun and a Hotel Bi...",2021,unknown,58,drama,United States,-1.0,tt10661500,4.3,145,6,6.3,Amazon,"Bradley Gosnell, Daniel Floren, Mia Marcon, Al..."
296,tm573955,Spoken,MOVIE,A group of friends try to find balance in thei...,2021,R,74,"drama, romance, horror, thriller",United States,-1.0,tt6455590,2.4,150,147,4.0,Amazon,"Ryan Paynter, Steven Serbones, Chelsea Skalski..."


In [24]:
# 8. Filtro por tema, época o lugar favorito
def buscar_sinonimos(critica, sinonimos):
    for sinonimo in sinonimos:
        if sinonimo in critica:
            return True
    return False
    
if respuestas['pregunta8'] != '':
    tema = respuestas['pregunta8']
    comentarios_filtrados = comments[comments['review'].apply(lambda x: buscar_sinonimos(x, tema))]
    df_filtrado = df_filtrado.merge(comentarios_filtrados[['imdb_id']], on='imdb_id', how='inner')
else:
    pass

In [26]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm1179579,In Good Hands,MOVIE,"Diagnosed with a terminal illness, a single mo...",2022,unknown,104,"drama, comedy, romance, family",Turkey,-1.0,tt14898794,5.6,2618,829,6.7,Netflix,"Aslı Enver, Kaan Urgancıoğlu, Mert Ege Ak, Ezg..."
1,ts328194,My Name,SHOW,"Following her father's murder, a revenge-drive...",2021,TV-MA,50,"thriller, action, crime, drama","Korea, Republic of",1.0,tt12940504,7.8,19737,86286,8.3,Netflix,"Han So-hee, Park Hee-soon, Ahn Bo-hyun, Kim Sa..."
2,tm974552,Last Three Days,MOVIE,After getting mixed up with a dangerous crime ...,2020,R,87,"thriller, action, crime",United States,-1.0,tt6794504,3.6,573,692,6.7,Amazon,"Robert Palmer Watkins, Thomas Wilson Brown, De..."
3,ts297675,Sexy Beasts,SHOW,"Hoping to say goodbye to superficial dating, r...",2021,TV-14,23,"reality, romance",unknown,2.0,tt14916746,4.6,1133,36721,5.9,Netflix,Rob Delaney
4,tm171793,Monsieur Lazhar,MOVIE,"During a harsh Montréal winter, an elementary-...",2012,PG-13,94,"drama, comedy",Canada,-1.0,tt2011971,7.5,20735,7152,7.1,Amazon,"Mohamed Fellag, Émilien Néron, Danielle Proulx..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,tm1031371,One and the Same,MOVIE,"When a young woman, Lola, is invited to a secr...",2021,unknown,110,scifi,United States,-1.0,tt8906844,4.3,4965,1011,5.7,Amazon,"Acoryé White, Glenn Plummer, Ignacyo Matynia, ..."
294,tm552220,The Blackout Experiment,MOVIE,6 strangers wake up inside a room garnished wi...,2021,R,81,"horror, thriller, drama",United States,-1.0,tt3659284,2.9,239,6395,5.0,Amazon,"Yasmin Irvine, Stefani Rose, Uli Auliani, Chev..."
295,tm822572,Gun and a Hotel Bible,MOVIE,"Based on the original play, Gun and a Hotel Bi...",2021,unknown,58,drama,United States,-1.0,tt10661500,4.3,145,6,6.3,Amazon,"Bradley Gosnell, Daniel Floren, Mia Marcon, Al..."
296,tm573955,Spoken,MOVIE,A group of friends try to find balance in thei...,2021,R,74,"drama, romance, horror, thriller",United States,-1.0,tt6455590,2.4,150,147,4.0,Amazon,"Ryan Paynter, Steven Serbones, Chelsea Skalski..."


In [27]:
# 9. Filtro por plataforma
if respuestas['pregunta9'] != '':
    plat_favorito = respuestas['pregunta9']
    df_filtrado = df_filtrado[df_filtrado['platform'].str.contains(plat_favorito, case=False)]
else:
    pass


In [28]:
df_filtrado

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_votes,tmdb_score,platform,actors
0,tm1179579,In Good Hands,MOVIE,"Diagnosed with a terminal illness, a single mo...",2022,unknown,104,"drama, comedy, romance, family",Turkey,-1.0,tt14898794,5.6,2618,829,6.7,Netflix,"Aslı Enver, Kaan Urgancıoğlu, Mert Ege Ak, Ezg..."
1,ts328194,My Name,SHOW,"Following her father's murder, a revenge-drive...",2021,TV-MA,50,"thriller, action, crime, drama","Korea, Republic of",1.0,tt12940504,7.8,19737,86286,8.3,Netflix,"Han So-hee, Park Hee-soon, Ahn Bo-hyun, Kim Sa..."
3,ts297675,Sexy Beasts,SHOW,"Hoping to say goodbye to superficial dating, r...",2021,TV-14,23,"reality, romance",unknown,2.0,tt14916746,4.6,1133,36721,5.9,Netflix,Rob Delaney
6,ts223242,Was It Love?,SHOW,"After 14 years devoid of romance, a struggling...",2020,TV-MA,69,"drama, comedy, romance","Korea, Republic of",1.0,tt12525622,7.1,907,4721,8.1,Netflix,"Song Ji-hyo, Son Ho-jun, Song Jong-ho, Kim Min..."
8,ts310938,New Heights,SHOW,When business consultant Michi Wyss inherits h...,2021,TV-MA,48,drama,Switzerland,1.0,tt15393644,6.7,525,8664,7.9,Netflix,"Julian Koechlin, Sophie Hutter, Anouk Petri, J..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,tm361414,The Guilty,MOVIE,"Police officer Asger Holm, demoted to desk wor...",2018,R,85,"thriller, crime, drama, european",Denmark,-1.0,tt6742252,7.5,60804,12353,7.3,"Netflix, HBO","Jakob Cedergren, Jessica Dinnage, Omar Shargaw..."
251,tm228223,The Innocents,MOVIE,"Poland, 1945. Mathilde, a young French Red Cro...",2016,PG-13,115,"drama, history, european","Belgium, France, Poland",-1.0,tt4370784,7.3,10518,10035,7.1,"Netflix, Amazon","Lou de Laâge, Agata Buzek, Agata Kulesza, Vinc..."
264,tm317586,Paradox,MOVIE,When a Hong Kong police negotiator is informed...,2017,R,101,"action, drama, crime, thriller","China, Hong Kong",-1.0,tt7245176,6.5,4109,18504,6.6,"Netflix, Amazon","Louis Koo, Wu Yue, Gordon Lam, Chris Collins, ..."
265,tm438262,Unstoppable,MOVIE,Dong-chul and Ji-soo are a happily married cou...,2018,R,115,"crime, thriller, action, european","Korea, Republic of",-1.0,tt9225192,6.6,3740,13161,7.6,"Netflix, Amazon","Ma Dong-seok, Song Ji-hyo, Kim Sung-oh, Kim Mi..."
